## Data Cleaning

In [1]:
# Imports
import pandas as pd
import numpy as np
import requests

In [2]:
# Using requests to get the data in json form
scrape = requests.get('https://www.levels.fyi/js/salaryData.json').json()

# Turning the json into a pandas dataframe
df = pd.DataFrame(scrape)

# Changing the columns from type object to numeric types and datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df[['yearsofexperience','basesalary','bonus','stockgrantvalue','totalyearlycompensation','yearsatcompany']] = df[['yearsofexperience','basesalary','bonus','stockgrantvalue','totalyearlycompensation','yearsatcompany']].apply(pd.to_numeric)

# Only keeping entries with compensation given
df = df[df['totalyearlycompensation'] > 0]

# Intern Salaries can throw off the averages
df = df[df['level'] != "Intern"]

# If the entry is from the USA then there is only one comma in the location (i.e. San Francisco, CA)
df = df[df['location'].str.count(',') <= 1]

# Data from before 2020 has some differences (the form didn't ask for amounts in thousands) and may not reflect current salary amounts
df = df[df['timestamp'] >= ('2020-01-01')]

# Cleaning by: 
## Dropping Unneeded columns, turning missing values into NaNs, sorting by compensation, adding a state column, and normalizing company column
df = df.drop(['cityid', 'dmaid','rowNumber', 'otherdetails'], axis=1)
df = df.replace("", np.nan)
df = df.sort_values(by='totalyearlycompensation')
df["state"] = df["location"].str[-2:] 
df["company"] = df["company"].str.title()

# Reordering Columns
df = df[['timestamp', 'company', 'level', 'title', 'totalyearlycompensation',
       'location', 'state', 'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender']]

# The form on levels.fyi asks for the quantities in thousands
cols = ["totalyearlycompensation", "basesalary",'bonus','stockgrantvalue']
for col in cols:
    df[col] = df[col]*1000

In [3]:
df

,timestamp,company,level,title,totalyearlycompensation,location,state,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender
46802,2021-03-25 06:11:09,Delphix,entry level,Software Engineer,8000,"San Francisco, CA",CA,0,0,API Development (Back-End),7000,0,1000,NaN
25758,2020-07-25 23:28:10,Verkada,l3,Marketing,11000,"San Francisco, CA",CA,2,1,Marketing,11000,0,0,Female
38651,2020-12-21 13:44:40,Ansys,L1,Software Engineer,12000,"Canonsburg, PA",PA,1,1,Testing (SDET),12000,0,0,Male
58340,2021-07-13 11:47:35,Accenture,11,Data Scientist,14000,"Indianapolis, IN",IN,0,0,Data,10000,0,4000,Male
60030,2021-07-27 19:26:24,General Dynamics Mission Systems,L3,Software Engineer,14000,"San Francisco, CA",CA,7,3,Android,14000,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56323,2021-06-24 19:29:24,Uber,Sr Director,Product Manager,2641000,"San Francisco, CA",CA,28,8,Product,446000,1989000,206000,NaN
52595,2021-05-18 15:34:21,Facebook,D1,Software Engineering Manager,2702000,"Menlo Park, CA",CA,33,1,ML / AI,192000,2411000,99000,Male
45234,2021-03-09 17:03:07,Google,L10,Product Manager,3656000,"San Francisco, CA",CA,15,10,Consumer,302000,3062000,292000,Male
57241,2021-07-02 08:43:37,Facebook,E9,Software Engineer,4680000,"Menlo Park, CA",CA,11,5,Full Stack,221000,4268000,191000,NaN
